## The second step is completing the 4-step IBC connection handshake between terra and osmosis - 1) init (on inj), 2) try (on osmosis), 3) ack (on inj), 4) confirm (on osmosis).

## With an IBC connection in-place, we lay the foundation for injective/osmosis smart contract pairs to communicate.

In [2]:
#imports

import pandas as pd
import json
import os
import sys
import base64
import requests
import subprocess
import math
import hashlib
import bech32
import time

from dateutil.parser import parse
from datetime import datetime, timedelta
from ecdsa import SECP256k1, SigningKey
from ecdsa.util import sigencode_string_canonize
from bech32 import bech32_decode, bech32_encode, convertbits
from google.protobuf.timestamp_pb2 import Timestamp as googTimestamp

from terra_sdk.client.lcd import LCDClient
from terra_sdk.core.wasm import MsgStoreCode, MsgInstantiateContract, MsgExecuteContract
from terra_sdk.core.bank import MsgSend
from terra_sdk.core.fee import Fee
from terra_sdk.key.mnemonic import MnemonicKey
from terra_sdk.core.bech32 import get_bech
from terra_sdk.core import AccAddress, Coin, Coins
from terra_sdk.client.lcd.api.tx import CreateTxOptions, SignerOptions
from terra_sdk.client.localterra import LocalTerra
from terra_sdk.core.wasm.data import AccessConfig
from terra_sdk.client.lcd.api._base import BaseAsyncAPI, sync_bind

from terra_proto.cosmwasm.wasm.v1 import AccessType
from terra_proto.cosmos.tx.v1beta1 import Tx, TxBody, AuthInfo, SignDoc, SignerInfo, ModeInfo, ModeInfoSingle, BroadcastTxResponse
from terra_proto.cosmos.base.abci.v1beta1 import TxResponse
from terra_proto.cosmos.tx.signing.v1beta1 import SignMode
from terra_proto.ibc.core.client.v1 import MsgCreateClient, Height, MsgUpdateClient, QueryClientStateRequest, QueryClientStateResponse
from terra_proto.ibc.core.channel.v1 import MsgChannelOpenInit, Channel, State, Order, Counterparty, MsgChannelOpenTry, MsgChannelOpenAck, MsgChannelOpenConfirm, QueryUnreceivedPacketsRequest, QueryUnreceivedPacketsResponse, QueryPacketCommitmentRequest, QueryPacketCommitmentResponse, Packet, QueryNextSequenceReceiveRequest, QueryNextSequenceReceiveResponse, MsgRecvPacket, MsgTimeout, QueryUnreceivedAcksRequest, QueryUnreceivedAcksResponse, MsgAcknowledgement
from terra_proto.ibc.core.connection.v1 import MsgConnectionOpenInit, Counterparty as ConnectionCounterParty, Version, MsgConnectionOpenTry, MsgConnectionOpenAck, MsgConnectionOpenConfirm
from terra_proto.ibc.lightclients.tendermint.v1 import ClientState, ConsensusState, Fraction, Header
from terra_proto.ics23 import HashOp, LengthOp, LeafOp, InnerOp, ProofSpec, InnerSpec, CommitmentProof, ExistenceProof, NonExistenceProof, BatchProof, CompressedBatchProof, BatchEntry, CompressedBatchEntry, CompressedExistenceProof, CompressedNonExistenceProof
from terra_proto.ibc.core.commitment.v1 import MerkleRoot, MerklePrefix, MerkleProof
from terra_proto.tendermint.types import ValidatorSet, Validator, SignedHeader, Header as tendermintHeader, Commit, BlockId, PartSetHeader, CommitSig, BlockIdFlag
from terra_proto.tendermint.version import Consensus
from terra_proto.tendermint.crypto import PublicKey
from betterproto.lib.google.protobuf import Any
from betterproto import Timestamp

#misc helper functions
sys.path.append(os.path.join(os.path.dirname(__name__), '..', 'scripts'))

from helpers import proto_to_binary, timestamp_string_to_proto, stargate_msg, create_ibc_client, fetch_chain_objects, bech32_to_hexstring, hexstring_to_bytes, bech32_to_b64, b64_to_bytes, fabricate_update_client, fetch_proofs

In [3]:
#setup lcd clients, rpc urls, wallets
(inj, inj_wallet, inj_rpc_url, inj_rpc_header) = fetch_chain_objects("injective-888")
(osmo, osmo_wallet, osmo_rpc_url, osmo_rpc_header) = fetch_chain_objects("osmo-test-4")

## These are the protobuf classes specific for setting up the IBC connection. One noteworthy protobuf class that will see continual usage now, and in later stages, is MsgUpdateClient, which basically refreshes the light client information (ie the counterpart chain tendermint state).

In [4]:
#terra_proto imports
from terra_proto.ibc.core.connection.v1 import MsgConnectionOpenInit, Counterparty as ConnectionCounterParty, Version, MsgConnectionOpenTry, MsgConnectionOpenAck, MsgConnectionOpenConfirm
from terra_proto.ibc.core.client.v1 import MsgCreateClient, Height, MsgUpdateClient, QueryClientStateRequest, QueryClientStateResponse

## Load client_id's from previous step; if we don't find it, we'll use default client_id's built by the authors.

In [5]:
#load context.json
context = {}
# Open the file for reading
with open("context.json", "r") as f:
    # Load the dictionary from the file
    context = json.load(f)
    
client_id_on_inj = context["client_id_on_inj"]
client_id_on_osmo = context["client_id_on_osmo"]

print(context)

{'client_id_on_inj': '07-tendermint-87', 'client_id_on_osmo': '07-tendermint-4298'}


## IBC connection step 1 - MsgConnectionOpenInit (called on injective)

In [6]:
#fabricate message & dispatch via stargate
msg = MsgConnectionOpenInit(
  client_id=client_id_on_inj,
  counterparty=ConnectionCounterParty(
    client_id=client_id_on_osmo,
    prefix=MerklePrefix(
      key_prefix=bytes("ibc", "ascii"),
    )
  ),
  version=Version(
    identifier="1",
    features=["ORDER_ORDERED", "ORDER_UNORDERED"],
  ),
  delay_period=0,
  signer=inj_wallet.key.acc_address
)

result = stargate_msg("/ibc.core.connection.v1.MsgConnectionOpenInit", msg, inj_wallet, inj)
result_df = pd.DataFrame(result["tx_response"]["logs"][0]["events"][0]["attributes"])
connection_id_on_inj = result_df[result_df["key"]=="connection_id"]["value"].values[0]

print(f"connection_id on inj: {connection_id_on_inj}")

connection_id on inj: connection-69


## IBC connection step 2 - MsgConnectionOpenTry (called on osmosis)

In [7]:
#update the client_id on osmosis (ie, refresh osmosis's knowledge of injectives's tendermint state)
#only this cell will show the full gory details; the rest will just call the helper function

#gather tendermint/consensus/validator information on injective's current state
time.sleep(10) #wait a few blocks
tendermint_info_on_other_chain = inj.tendermint.block_info()

timestamp = googTimestamp()
timestamp.FromJsonString(tendermint_info_on_other_chain["block"]["header"]["time"])
print(f"timestamp: {timestamp} \n\n")

print("source current tendermint:")
print(tendermint_info_on_other_chain)
print("\n\n")

validator_info_on_other_chain = inj.tendermint.validator_set(height=int(tendermint_info_on_other_chain["block"]["header"]["height"]))
commit_info_on_other_chain = requests.get(f"{inj_rpc_url}/commit", headers=inj_rpc_header, params={"height": tendermint_info_on_other_chain["block"]["header"]["height"]}).json()

print("\n\ncommit_info:\n")
print(commit_info_on_other_chain)
print("\n\n")

resp = requests.get(f"{inj_rpc_url}/blockchain", headers=inj_rpc_header, params={"minHeight": tendermint_info_on_other_chain["block"]["header"]["height"], "maxHeight": tendermint_info_on_other_chain["block"]["header"]["height"]}).json()
block_proposer_on_other_chain = resp["result"]["block_metas"][0]["header"]["proposer_address"]

proposer_info = None
for x in validator_info_on_other_chain["validators"]:
    if block_proposer_on_other_chain == bech32_to_hexstring(x["address"]).upper():
        proposer_info = x

#gather osmosis's understanding of inj's tendermint/consensus/validator information 
client_state_of_other_chain_on_my_chain = osmo.broadcaster.query(f"/ibc/core/client/v1/client_states/{client_id_on_osmo}")
validator_info_of_other_chain_on_my_chain = inj.tendermint.validator_set(height=int(client_state_of_other_chain_on_my_chain["client_state"]["latest_height"]["revision_height"])+1)
block_proposer_of_other_chain_on_my_chain = requests.get(f"{inj_rpc_url}/blockchain", headers=inj_rpc_header, params={"minHeight": int(client_state_of_other_chain_on_my_chain["client_state"]["latest_height"]["revision_height"])+1, "maxHeight": int(client_state_of_other_chain_on_my_chain["client_state"]["latest_height"]["revision_height"])+1}).json()["result"]["block_metas"][0]["header"]["proposer_address"]

trusted_proposer_info = None
for x in validator_info_of_other_chain_on_my_chain["validators"]:
    if block_proposer_of_other_chain_on_my_chain == bech32_to_hexstring(x["address"]).upper():
        trusted_proposer_info = x

version_app = 0 if "app" not in commit_info_on_other_chain["result"]["signed_header"]["header"]["version"].keys() else int(commit_info_on_other_chain["result"]["signed_header"]["header"]["version"]["app"])

#fabricate the MsgUpdateClient message; this updates osmosis's knowledge of inj's tendermint/consensus state
msg = MsgUpdateClient(
    signer=osmo_wallet.key.acc_address,
    client_id=client_id_on_osmo,
    header=Any(
      type_url="/ibc.lightclients.tendermint.v1.Header",
      value=Header(
        signed_header=SignedHeader(
          header=tendermintHeader(
            version=Consensus(
              block=int(commit_info_on_other_chain["result"]["signed_header"]["header"]["version"]["block"]),
              app=version_app,
            ),
            chain_id=inj.chain_id,
            height=int(commit_info_on_other_chain["result"]["signed_header"]["header"]["height"]),
            time=Timestamp(timestamp.seconds, timestamp.nanos),
            last_block_id=BlockId(
              hash=hexstring_to_bytes(commit_info_on_other_chain["result"]["signed_header"]["header"]["last_block_id"]["hash"]),
              part_set_header=PartSetHeader(
                total=int(commit_info_on_other_chain["result"]["signed_header"]["header"]["last_block_id"]["parts"]["total"]),
                hash=hexstring_to_bytes(commit_info_on_other_chain["result"]["signed_header"]["header"]["last_block_id"]["parts"]["hash"]),
              ),
            ),
            last_commit_hash=hexstring_to_bytes(commit_info_on_other_chain["result"]["signed_header"]["header"]["last_commit_hash"]),
            data_hash=hexstring_to_bytes(commit_info_on_other_chain["result"]["signed_header"]["header"]["data_hash"]),
            validators_hash=hexstring_to_bytes(commit_info_on_other_chain["result"]["signed_header"]["header"]["validators_hash"]),
            next_validators_hash=hexstring_to_bytes(commit_info_on_other_chain["result"]["signed_header"]["header"]["next_validators_hash"]),
            consensus_hash=hexstring_to_bytes(commit_info_on_other_chain["result"]["signed_header"]["header"]["consensus_hash"]),
            app_hash=hexstring_to_bytes(commit_info_on_other_chain["result"]["signed_header"]["header"]["app_hash"]),
            last_results_hash=hexstring_to_bytes(commit_info_on_other_chain["result"]["signed_header"]["header"]["last_results_hash"]),
            evidence_hash=hexstring_to_bytes(commit_info_on_other_chain["result"]["signed_header"]["header"]["evidence_hash"]),
            proposer_address=hexstring_to_bytes(commit_info_on_other_chain["result"]["signed_header"]["header"]["proposer_address"]),
          ),
          commit=Commit(
            height=int(commit_info_on_other_chain["result"]["signed_header"]["commit"]["height"]),
            round=int(commit_info_on_other_chain["result"]["signed_header"]["commit"]["round"]),
            block_id=BlockId(
              hash=hexstring_to_bytes(commit_info_on_other_chain["result"]["signed_header"]["commit"]["block_id"]["hash"]),
              part_set_header=PartSetHeader(
                total=int(commit_info_on_other_chain["result"]["signed_header"]["commit"]["block_id"]["parts"]["total"]),
                hash=hexstring_to_bytes(commit_info_on_other_chain["result"]["signed_header"]["commit"]["block_id"]["parts"]["hash"]),
              )
            ),
            signatures=[
              CommitSig(
                block_id_flag=BlockIdFlag(x["block_id_flag"]),
                validator_address=hexstring_to_bytes(x["validator_address"]),
                timestamp=timestamp_string_to_proto(x["timestamp"]),
                signature=base64.b64decode(x["signature"]) if x["signature"] is not None else None,
              )
              for x in commit_info_on_other_chain["result"]["signed_header"]["commit"]["signatures"]
            ],
          ),
        ),
        validator_set=ValidatorSet(
          validators=[
            Validator(
              address=base64.b64decode(bech32_to_b64(x["address"])),
              pub_key=PublicKey(
                ed25519=base64.b64decode(x["pub_key"]["key"])
              ) if "ed25519" in x["pub_key"]["@type"] else PublicKey(
                secp256_k1=base64.b64decode(x["pub_key"]["key"])
              ),
              voting_power=int(x["voting_power"]),
            ) for x in validator_info_on_other_chain["validators"]
          ],
          proposer=Validator(
            address=base64.b64decode(bech32_to_b64(proposer_info["address"])),
            pub_key=PublicKey(
              ed25519=base64.b64decode(proposer_info["pub_key"]["key"])
            ) if "ed25519" in proposer_info["pub_key"]["@type"] else PublicKey(
                secp256_k1=base64.b64decode(proposer_info["pub_key"]["key"])
            ),
            voting_power=int(proposer_info["voting_power"]),
          ) 
          ,
          total_voting_power=sum([int(x["voting_power"]) for x in validator_info_on_other_chain["validators"]]),
        ),
        trusted_height=Height(
          revision_number=int(client_state_of_other_chain_on_my_chain["client_state"]["latest_height"]["revision_number"]),
          revision_height=int(client_state_of_other_chain_on_my_chain["client_state"]["latest_height"]["revision_height"]),
        ),
        trusted_validators=ValidatorSet(
          validators=[
            Validator(
              address=base64.b64decode(bech32_to_b64(x["address"])),
              pub_key=PublicKey(
                ed25519=base64.b64decode(x["pub_key"]["key"])
              ) if "ed25519" in x["pub_key"]["@type"] else PublicKey(
                secp256_k1=base64.b64decode(x["pub_key"]["key"])
              ),
              voting_power=int(x["voting_power"]),
            ) for x in validator_info_of_other_chain_on_my_chain["validators"]
          ],
          proposer=Validator(
            address=base64.b64decode(bech32_to_b64(trusted_proposer_info["address"])),
            pub_key=PublicKey(
              ed25519=base64.b64decode(trusted_proposer_info["pub_key"]["key"])
            ) if "ed25519" in trusted_proposer_info["pub_key"]["@type"] else PublicKey(
                secp256_k1=base64.b64decode(trusted_proposer_info["pub_key"]["key"])
            ),
            voting_power=int(trusted_proposer_info["voting_power"]),
          ) 
          ,
          total_voting_power=sum([int(x["voting_power"]) for x in validator_info_of_other_chain_on_my_chain["validators"]]),
        ),
      ).SerializeToString(),
    ),
)

print(f"""

update client msg:

{ msg.to_dict()}

""")

update_client_on_osmo_result = stargate_msg("/ibc.core.client.v1.MsgUpdateClient", msg, osmo_wallet, osmo)
header_height = Header.FromString(msg.header.value).signed_header.header.height

timestamp: seconds: 1678825745
nanos: 847033536
 


source current tendermint:
{'block_id': {'hash': 'XIM1hpeo3emqNzK+DXUfuXJaUwTm96+DpVoQCa8L4vY=', 'part_set_header': {'total': 1, 'hash': 'YUyEeISIpOd/QEtrNS2H+ZZPfhPRsPAvJIUt2btdWfA='}}, 'block': {'header': {'version': {'block': '11', 'app': '0'}, 'chain_id': 'injective-888', 'height': '9199704', 'time': '2023-03-14T20:29:05.847033536Z', 'last_block_id': {'hash': 'e62UW4cOA/d2ukcSxrL44zOljxcTnwxxMqlX+XrClRw=', 'part_set_header': {'total': 1, 'hash': 'G1jPhCFwg7wabU9P9VXFUT0ovvp5RsKvcUzGLxkpSHQ='}}, 'last_commit_hash': 'mXvqMIlwFgYViGSmMOcutqIL3Cw12u+sADSxtqWE2uo=', 'data_hash': 'K91Eeztd8UmVQvW2fJETK7mkWQqNOxEJNSPm145XDWY=', 'validators_hash': 'FI/e4hLK65ZxQjpH7coHwFCKHMb3WPVzA5PGSyB7KMg=', 'next_validators_hash': 'FI/e4hLK65ZxQjpH7coHwFCKHMb3WPVzA5PGSyB7KMg=', 'consensus_hash': 'vml94SL+cPwUT5B4L5ZAe/3n0gSLsv2tqMpUVEVA8gg=', 'app_hash': 'v2WSM1d25S97PG1hucEjAsquNkPOdtjRrxlOQ5CWJQU=', 'last_results_hash': '47DEQpj8HBSa+/TImW+5JCeuQeRk

In [8]:
#fetch client, connection, consensus proofs

inj_client_trusted_height = header_height
inj_client_trusted_revision_number = osmo.broadcaster.query(f"/ibc/core/client/v1/client_states/{client_id_on_osmo}")["client_state"]["latest_height"]["revision_number"]

#client state proof - inj's understanding of osmo's state
params = {
    "path": '"/store/ibc/key"',
    "data": "0x" + bytes(f"clients/{client_id_on_inj}/clientState", "ascii").hex(),
    "prove": "true",
    "height": int(inj_client_trusted_height) - 1,
}
osmo_client_proof_on_inj = requests.get(f"{inj_rpc_url}/abci_query", headers=inj_rpc_header, params=params).json()
proofs = [CommitmentProof.FromString(b64_to_bytes(x["data"])) for x in osmo_client_proof_on_inj["result"]["response"]["proofOps"]["ops"]]
client_proof = MerkleProof(proofs=proofs)

print(f"""
client_proof:

{client_proof}

""")

#connection proof
time.sleep(2)
params = {
    "path": '"/store/ibc/key"',
    "data": "0x" + bytes(f"connections/{connection_id_on_inj}", "ascii").hex(),
    "prove": "true",
    "height": int(inj_client_trusted_height) - 1,
}
connection_proof_on_inj = requests.get(f"{inj_rpc_url}/abci_query", headers=inj_rpc_header, params=params).json()
connection_proofs = [CommitmentProof.FromString(b64_to_bytes(x["data"])) for x in connection_proof_on_inj["result"]["response"]["proofOps"]["ops"]]
connection_proof = MerkleProof(proofs=connection_proofs)

print(f"""
connection_proof:

{connection_proof}

""")

#fetch inj's understanding of osmo's consensus height 
time.sleep(3)
params = {
    "path": '"/ibc.core.client.v1.Query/ClientState"',
    "data": "0x" + QueryClientStateRequest(client_id_on_inj).SerializeToString().hex(),
    "prove": "false",
}
consensus_height = ClientState.FromString(QueryClientStateResponse.FromString(b64_to_bytes(requests.get(f"{inj_rpc_url}/abci_query", headers=inj_rpc_header, params=params).json()["result"]["response"]["value"])).client_state.value).latest_height

print(f"""
consensus_height:

{consensus_height}

""")

#consensus proof - inj's understanding of osmo's consensus
time.sleep(3)
params = {
    "path": '"/store/ibc/key"',
    "data": "0x" + bytes(f"clients/{client_id_on_inj}/consensusStates/{consensus_height.revision_number}-{consensus_height.revision_height}", "ascii").hex(),
    "prove": "true",
    "height": int(inj_client_trusted_height) - 1,
}
consensus_proof_on_inj = requests.get(f"{inj_rpc_url}/abci_query", headers=inj_rpc_header, params=params).json()
consensus_proofs = [CommitmentProof.FromString(b64_to_bytes(x["data"])) for x in consensus_proof_on_inj["result"]["response"]["proofOps"]["ops"]]
consensus_proof = MerkleProof(proofs=consensus_proofs)

print(f"""
consensus_proof:

{consensus_proof}

""")


client_state = Any(
    type_url="/ibc.lightclients.tendermint.v1.ClientState",
    value=ClientState.FromString(
      Any.FromString(
        b64_to_bytes(osmo_client_proof_on_inj["result"]["response"]["value"])
      ).value
    ).SerializeToString()
)




client_proof:

MerkleProof(proofs=[CommitmentProof(exist=ExistenceProof(key=b'clients/07-tendermint-87/clientState', value=b'\n+/ibc.lightclients.tendermint.v1.ClientState\x12\x83\x01\n\x0bosmo-test-4\x12\x04\x08\x01\x10\x03\x1a\x04\x08\x80\x9c1"\x04\x08\x80\xeaI*\x02\x08\x142\x00:\x07\x08\x04\x10\xca\xba\xc7\x04B\x19\n\t\x08\x01\x18\x01 \x01*\x01\x00\x12\x0c\n\x02\x00\x01\x10!\x18\x04 \x0c0\x01B\x19\n\t\x08\x01\x18\x01 \x01*\x01\x00\x12\x0c\n\x02\x00\x01\x10 \x18\x01 \x010\x01J\x07upgradeJ\x10upgradedIBCStateP\x01X\x01', leaf=LeafOp(hash=1, prehash_value=1, length=1, prefix=b'\x00\x02\x9e\xfd\xe2\x08'), path=[InnerOp(hash=1, prefix=b'\x02\x04\xdc\x80\xe3\x08 ', suffix=b' V?a\xf8\x9a\x81i\x929\x8e\x19\xae\x1cV\xd6\x13\x13\xa6\x8b\xd4\xa3\xdb\xe0\xa4A\xbe\xf7\xb5+\xa4\x84\xaa'), InnerOp(hash=1, prefix=b'\x04\x08\xdc\x80\xe3\x08 ', suffix=b' \x85e{\xff\xaa\xdb\x1c5\xdb\xdb\xf4\x1c\xd7\x08\xcf>\x8c\xe4G\xd0X\xaf\x01\x07\xd4$w~\xc8\xe54\xeb'), InnerOp(hash=1, prefix=b'\x06\x10\xdc\x80\xe3

In [9]:
#fabricate message & dispatch via stargate

msg = MsgConnectionOpenTry(
  client_id=client_id_on_osmo, #str
  client_state=client_state, #Any
  counterparty=ConnectionCounterParty(
    client_id=client_id_on_inj,
    prefix=MerklePrefix(
      key_prefix=bytes("ibc", "ascii"),
    ),
    connection_id=connection_id_on_inj,
  ),
  delay_period=0, #int
  counterparty_versions=[
        Version(
          identifier="1",
          features=["ORDER_ORDERED", "ORDER_UNORDERED"],
        )
    ], #list of Version
  proof_height=Height(int(inj_client_trusted_revision_number), int(inj_client_trusted_height)), #Height
  proof_init=connection_proof.SerializeToString(), #bytes
  proof_client=client_proof.SerializeToString(), #bytes
  proof_consensus=consensus_proof.SerializeToString(), #bytes
  consensus_height=consensus_height, #Height
  signer=osmo_wallet.key.acc_address, #str
)

connection_try_on_osmo_result = stargate_msg("/ibc.core.connection.v1.MsgConnectionOpenTry", msg, osmo_wallet, osmo)
create_connection_on_osmo_df = pd.DataFrame(connection_try_on_osmo_result["tx_response"]["logs"][0]["events"][0]["attributes"])
connection_id_on_osmo = create_connection_on_osmo_df[create_connection_on_osmo_df["key"]=="connection_id"]["value"].values[0]

print(create_connection_on_osmo_df)

                          key               value
0               connection_id     connection-3714
1                   client_id  07-tendermint-4298
2      counterparty_client_id    07-tendermint-87
3  counterparty_connection_id       connection-69


## IBC connection step 3 - MsgConnectionOpenAck (called on inj)

In [10]:
#update client on inj (ie, refresh inj's knowledge of osmosis's tendermint state) & fetch proofs

time.sleep(10) #wait a few blocks
msg = fabricate_update_client(osmo, osmo_rpc_url, osmo_rpc_header, inj, inj_wallet, client_id_on_inj)
update_client_on_inj_result = stargate_msg("/ibc.core.client.v1.MsgUpdateClient", msg, inj_wallet, inj)
header_height = Header.FromString(msg.header.value).signed_header.header.height

osmo_client_trusted_height = header_height
osmo_client_trusted_revision_number = inj.broadcaster.query(f"/ibc/core/client/v1/client_states/{client_id_on_inj}")["client_state"]["latest_height"]["revision_number"]

client_proof, connection_proof, consensus_proof, consensus_height, client_state = fetch_proofs(osmo_rpc_url, osmo_rpc_header, client_id_on_osmo, osmo_client_trusted_height, osmo_client_trusted_revision_number, connection_id_on_osmo)

{'remote_lcd': <terra_sdk.client.lcd.lcdclient.LCDClient object at 0x7f234047da00>, 'remote_rpc_url': 'https://rpc-test.osmosis.zone', 'remote_rpc_header': {}, 'client_lcd': <terra_sdk.client.lcd.lcdclient.LCDClient object at 0x7f2340052580>, 'client_wallet': <terra_sdk.client.lcd.wallet.Wallet object at 0x7f234041fa00>, 'client_id': '07-tendermint-87'}



timestamp: seconds: 1678825871
nanos: 455350963
 


source current tendermint:
{'block_id': {'hash': 'zSR0S3nv79QS4yTDpLXhlfO7MfOr83/TNRpNvpC9Qe8=', 'part_set_header': {'total': 1, 'hash': 'TSoAXHuheMUHifRajBVzIF8X+YlI9C7YnkDJ3IwKXz4='}}, 'block': {'header': {'version': {'block': '11', 'app': '16'}, 'chain_id': 'osmo-test-4', 'height': '9559525', 'time': '2023-03-14T20:31:11.455350963Z', 'last_block_id': {'hash': '2l33FWPW/tkfRTFtBksvG2pss/NzzwDlyD9B0XK+ONI=', 'part_set_header': {'total': 1, 'hash': 'zzwLdaJ5vNfji31aEiRwJIrvSzbCqpfFRfJYSw/KyJc='}}, 'last_commit_hash': 'G4/kUmi0e0bu4XHAYaKNYMKpk2DCG9uyADMXS+fB6Wo=', 'data_hash': '47DE

In [12]:
#fabricate message & dispatch via stargate


msg = MsgConnectionOpenAck(
  connection_id=connection_id_on_inj,
  counterparty_connection_id=connection_id_on_osmo,
  version=Version(
      identifier="1",
      features=["ORDER_ORDERED", "ORDER_UNORDERED"],
    ),
  client_state=client_state,
  proof_height=Height(int(osmo_client_trusted_revision_number), int(osmo_client_trusted_height)),
  proof_try=connection_proof.SerializeToString(),
  proof_client=client_proof.SerializeToString(),
  proof_consensus=consensus_proof.SerializeToString(),
  consensus_height=consensus_height,
  signer=inj_wallet.key.acc_address,
)

connection_ack_on_inj_result = stargate_msg("/ibc.core.connection.v1.MsgConnectionOpenAck", msg, inj_wallet, inj)
connection_ack_on_inj_df = pd.DataFrame(connection_ack_on_inj_result["tx_response"]["logs"][0]["events"][0]["attributes"])

print(connection_ack_on_inj_df)

                          key               value
0               connection_id       connection-69
1                   client_id    07-tendermint-87
2      counterparty_client_id  07-tendermint-4298
3  counterparty_connection_id     connection-3714


## IBC connection step 4 - MsgConnectionOpenConfirm (called on osmosis)

In [13]:
#update client again on osmosis & fetch proofs

time.sleep(10) #wait a few blocks
msg = fabricate_update_client(inj, inj_rpc_url, inj_rpc_header, osmo, osmo_wallet, client_id_on_osmo)
update_client_on_osmo_result = stargate_msg("/ibc.core.client.v1.MsgUpdateClient", msg, osmo_wallet, osmo)
header_height = Header.FromString(msg.header.value).signed_header.header.height

inj_client_trusted_height = header_height
inj_client_trusted_revision_number = osmo.broadcaster.query(f"/ibc/core/client/v1/client_states/{client_id_on_osmo}")["client_state"]["latest_height"]["revision_number"]

client_proof, connection_proof, consensus_proof, consensus_height, client_state = fetch_proofs(inj_rpc_url, inj_rpc_header, client_id_on_inj, inj_client_trusted_height, inj_client_trusted_revision_number, connection_id_on_inj)

{'remote_lcd': <terra_sdk.client.lcd.lcdclient.LCDClient object at 0x7f2340052580>, 'remote_rpc_url': 'https://k8s.testnet.tm.injective.network:443/', 'remote_rpc_header': {}, 'client_lcd': <terra_sdk.client.lcd.lcdclient.LCDClient object at 0x7f234047da00>, 'client_wallet': <terra_sdk.client.lcd.wallet.Wallet object at 0x7f2340278cd0>, 'client_id': '07-tendermint-4298'}



timestamp: seconds: 1678825974
nanos: 296918484
 


source current tendermint:
{'block_id': {'hash': 'zt1aANCJzee3vV+AbGSmkFmpHrjb78ZGOf9D4sCToAk=', 'part_set_header': {'total': 1, 'hash': 'oIaQ1Y0gVDX+n5o9EJzYQl2gs7tCyLRq+L2T47E7+/w='}}, 'block': {'header': {'version': {'block': '11', 'app': '0'}, 'chain_id': 'injective-888', 'height': '9199798', 'time': '2023-03-14T20:32:54.296918484Z', 'last_block_id': {'hash': '6azuT5FAVtcbjGCrMADtABK8LshqejkaK7kNM/JVBdc=', 'part_set_header': {'total': 1, 'hash': 'rx3RqMG52Br2X0DUGrl3s/Rth8MNYh7zsxzNSdoLc64='}}, 'last_commit_hash': 'vSXyc9us6tzDDlVTAi171UXRYo+WZZU0EI6yXNpjesM=',

In [14]:
#fabricate message & dispatch via stargate


msg = MsgConnectionOpenConfirm(
  connection_id=connection_id_on_osmo,
  proof_ack=connection_proof.SerializeToString(),
  proof_height=Height(int(inj_client_trusted_revision_number), int(inj_client_trusted_height)),
  signer=osmo_wallet.key.acc_address,
)

connection_confirm_on_osmo_result = stargate_msg("/ibc.core.connection.v1.MsgConnectionOpenConfirm", msg, osmo_wallet, osmo)
connection_confirm_on_osmo_df = pd.DataFrame(connection_confirm_on_osmo_result["tx_response"]["logs"][0]["events"][0]["attributes"])

print(connection_confirm_on_osmo_df)

                          key               value
0               connection_id     connection-3714
1                   client_id  07-tendermint-4298
2      counterparty_client_id    07-tendermint-87
3  counterparty_connection_id       connection-69


In [15]:
#persist connection_id's on both chains for later stages

context["connection_id_on_inj"] = connection_id_on_inj
context["connection_id_on_osmo"] = connection_id_on_osmo
print(context)

with open("context.json", "w") as f:
    # Write the dictionary to the file as a JSON string
    json.dump(context, f)


{'client_id_on_inj': '07-tendermint-87', 'client_id_on_osmo': '07-tendermint-4298', 'connection_id_on_inj': 'connection-69', 'connection_id_on_osmo': 'connection-3714'}
